# Manipulations of dataframe

In [ ]:
def led_bins(dfg):
    zgroup_names=np.arange(-6,70,4)
    tbins=np.arange(0,196,2)
    if 'zs' in dfg.columns:
        zbins=np.arange(-8,72,4)
        dfg['zled'] = pd.cut(dfg['zs'], zbins, labels=zgroup_names).astype('Int64')
    if 'time' in dfg.columns:
        dfg['tled'] = pd.cut(dfg['frame'], tbins, labels=tgroup_names).astype('Int64')
    #if 'dist_c' in dfg.columns: 
     #   d
     #   
    #dfg['zled'] = pd.cut(dfg['zs'], zbins, labels=zgroup_names).astype('Int64') #210927:changed from 'zf'
    
    return dfg

In [ ]:
def new_vars(df):
#---------------------------------------------------------------------------
#Some general variables
    
    df['time']=df['frame']*3.1
    df.loc[:,'minute'] = pd.cut(df['time'], 10, labels=list(range(1,11,1)))
    df=df.rename(columns={'xs':'x_s'})
    track_thr=2
    df.loc[:,'tracked']=df.nrtracks>track_thr
    df['dist_c']=((df.loc[:,'x_s'])**2+(df.loc[:,'ys'])**2)**0.5
    df['dist_cz']=((df.loc[:,'x_s'])**2+(df.loc[:,'ys'])**2+(df.loc[:,'zs'])**2)**0.5
    #df['id']=df['exp_id'].astype('int').astype('str')+'-'+df['particle'].astype('str')
    
    return df
#---------------------------------------------------------------------------
def vars_expid(df):
# exp_id and inh_exp_id 

    for c,path in enumerate(pd.unique(df.path)):
        df.loc[(df.path==path),'exp_id']=c
#
    for x in pd.unique(df.inh): 
        num_exp_id=[]
        num_inh_exp_id=[]
        for n,y in enumerate (pd.unique(df[df.inh==x].exp_id),1):
            df.loc[(df.inh==x)&(df.exp_id==y),'inh_exp_id']=n
            num_exp_id.append(y)
            num_inh_exp_id.append(n)
        print(f'Treatment\t\t{x}\nNo of experiments\t{len(num_exp_id)}\nexp_id\t\t\t{num_exp_id}\ninh_exp_id\t\t{num_inh_exp_id}\n\n')
    return df



In [ ]:
def track_objs(df, search_range=3, memory=1):
    #tracked_df = tp.link_df(df, search_range, pos_columns=['xs', 'ys', 'zs'], memory=memory)

    df_tracks = []

    for i, grp in tracked_df.groupby(['particle']):
        grp=grp.sort_values('frame')
        dvx = np.append(np.diff(grp['xs']), np.nan)
        dvy = np.append(np.diff(grp['ys']), np.nan)
        dvz = np.append(np.diff(grp['zs']), np.nan)
        df_track=grp.pid.to_frame()
        #df_track['particle']=grp.particle
        df_track['nrtracks']=len(df_track)
        df_track['dvx']=dvx
        df_track['dvy']=dvy
        df_track['dvz']=dvz
        df_track['dv']=(df_track.dvx**2+df_track.dvy**2+df_track.dvz**2)**0.5
        df_tracks.append(df_track)

    df_tracks=pd.concat(df_tracks, axis=0)
    df_tracks=df_tracks.sort_values('pid')
    tracked_df = tracked_df.sort_values('pid')
    tracked_df = pd.concat([tracked_df, df_tracks[['nrtracks', 'dvx', 'dvy', 'dvz', 'dv']]], axis=1)
    return tracked_df

In [ ]:
#---------------------------------------------------------------------------

def vars_pos(df):
    #Skapar variabeln position som anger om trombocyten befinner sig i svansen eller huvudet
    position=['head','tail','outside']
    df['position']='outside'
    df.loc[(df.dist_c<38),'position']='head'
    df.loc[(df.dist_c>38)&(df.ys<0)&(abs(df.x_s)<38),'position']='tail'

    #Create variable which specifies if the platelet is inside or outside the laser injury zone
    df['inside_injury']=df.position.isin(['head'])
    #pc.loc[:,'pos_injury']='outside'
    #pc.loc[(pc.position=='head'),'bin_position']='inside'

    #sns.countplot(x='position',data=pc)
    print('Value counts Position\n',pc.position.value_counts(),'Value counts inside injury',df.inside_injury.value_counts())
    df.loc[:,'height']=pd.cut(df.zf, 3, labels=["bottom", "middle", "top"])
    df.loc[:,'z_pos']=pd.cut(df.zf, 2, labels=["bottom", "top"])
    
    df['zz']=np.round(df.loc[:,'zf'],decimals=0)#pc['zz']=np.round(pc.loc[:,'zs'],decimals=0)
    df = df.astype({'zz': int})
    zbins=np.arange(-8,72,4)
    zgroup_names=np.arange(-6,70,4)
    df['zled'] = pd.cut(df['zf'], zbins, labels=zgroup_names).astype('Int64')
    return df


#pc=pc.drop(columns='path')
    #return df


In [ ]:
#---------------------------------------------------------------------------
def vars_mov(df):
#Movement classification
    df.loc[:,'dvz_s']=df.dvz*1000/3.1
    df.loc[:,'cont_s']=df.loc[:,'cont']*1000/3.1
#Add categorical variable to describe platelet movement patterns
    #df.loc[:,'mov_class']='none'
    df.assign()

    for exp_id in pd.unique(df.exp_id):
        dfi=df[df.exp_id==exp_id].copy()
    
    #contractile=pd.unique(dfi[(dfi.cont_tot>5)&((dfi.cont_tot/dfi.displ_tot)>0.5)].particle)
    #New deifinition 191209
        try:
            still=pd.unique(dfi[((dfi.displ_tot/dfi.nrtracks)<0.1)&(dfi.displ_tot<4)].particle)
            loose=pd.unique(dfi[(dfi.displ_tot>5)&((dfi.cont_tot/dfi.displ_tot)<0.2)].particle)
            contractile=pd.unique(dfi[((dfi.cont_tot/dfi.displ_tot)>0.5)&(dfi.displ_tot>1)].particle)
        except TypeError:
            print(exp_id,dfi.displ_tot.dtypes,dfi.nrtracks.dtypes,(dfi.displ_tot/dfi.nrtracks))
    
        df.loc[(df.exp_id==exp_id)&(df['particle'].isin(still)),'mov_class']="still"
        df.loc[(df.exp_id==exp_id)&(df['particle'].isin(loose)),'mov_class']="loose"
        df.loc[(df.exp_id==exp_id)&(df['particle'].isin(contractile)),'mov_class']="contractile"


#Skapar variabeln movement som anger om trombocyterna for ogonblicket ror sig kontraktilt, driftar eller star stilla
    df['movement']='none'
    df.loc[(df.dv<0.1) & (df.tracked),'movement']='immobile' #pc.loc[(pc.dv)<0.3,'movement']='still'
    df.loc[(abs(df.dvy/df.dv)>0.5) & (df.dvy<0),'movement']='drifting' #pc.loc[(pc.dv>0.3)&(pc.cont_p<0.5),'movement']='drifting'
    df.loc[(df.dv>0.3) & (df.cont_p>0.5) ,'movement']='contracting'
    df.loc[(pc.stab>3),'movement']='unstable'
    return df



In [ ]:
def binning_labels(dfg,binned_var,bins):
    dfg[binned_var]=pd.cut(dfg[binned_var],bins,precision=0)
    bin_var=f'{binned_var}_binlabel'
    bin_labels=[]
    for bin in dfg[binned_var].sort_values().unique():
        bin_label=str(np.round(np.mean((bin.right+bin.left)/2),0))
        bin_labels.append(bin_label)
        #print (bin,bin_label)
        dfg.loc[dfg[binned_var]==bin,bin_var]=bin_label
    bin_order = sorted(bin_labels, key=lambda x: float(x))
    return dfg,bin_var,bin_order

In [ ]:
def qbinning_labels(dfg,binned_var,bins):
    dfg[binned_var]=pd.qcut(dfg[binned_var],bins,precision=0)
    bin_var=f'{binned_var}_binlabel'
    bin_labels=[]
    for bin in dfg[binned_var].sort_values().unique():
        bin_label=str(int(np.round(np.mean((bin.right+bin.left)/2),0)))
        bin_labels.append(bin_label)
        #print (bin,bin_label)
        dfg.loc[dfg[binned_var]==bin,bin_var]=bin_label
    bin_order = sorted(bin_labels, key=lambda x: float(x))
    return dfg,bin_var,bin_order

In [ ]:
def qbinning_labels1(dfg,binned_var,bins):
    dfg[binned_var]=pd.qcut(dfg[binned_var],bins,precision=0)
    bin_var=f'{binned_var}_binlabel'
    #bin_labels=[]
    for bin in dfg[binned_var].sort_values().unique():
        bin_label=int(np.round(np.mean((bin.right+bin.left)/2),0))
        #bin_labels.append(bin_label)
        #print (bin,bin_label)
        dfg.loc[dfg[binned_var]==bin,bin_var]=bin_label
    #bin_order = sorted(bin_labels, key=lambda x: float(x))
    return dfg,bin_var

# Customization tools

In [ ]:
def makedir(results_folder):
    try:
        os.mkdir(results_folder)
    
    except FileExistsError: 
        print(f'Folder {results_folder} already exists')

In [ ]:
def menu_var_choice():
    from IPython.display import clear_output    
#---------------------------------------------------------------------------------------------        
    def print_menu(dic_columns):       # Your menu design here
        #print(5 * "\n")
        print(73 * "-")
        print("Choose packages you don't need in dataframe")
        #print(73 * "-")
        print("1. Nearest neighbours: ",dic_columns[1])
        #print(73 * "-")
        print("2. Average dist nearest neighbours: ",dic_columns[2])
        #print(73 * "-")
        print("3. DBSCAN: ",dic_columns[3])
        #print(73 * "-")
        print("4. xtra tracking: ",dic_columns[4])
        #print(73 * "-")
        print("5. Platelet shape measures: ", dic_columns[5])
        #print(73 * "-")
        print("6. Fluorescence of plt marker c2: ", dic_columns[6])
        #print(73 * "-")
        print("7. Plt movement patterns: ", dic_columns[7])
        #print(73 * "-")
        print("8. Xtra plt position variables: ", dic_columns[8])
        #print(73 * "-")
        print("9. Scaled movements: ", dic_columns[8])
        
#--------------------------------------------------------------------------------------------- 

    def choices(columns,n_df,a,dic_columns):
       
                    
        #columns=pc.columns
        #if n_df==0:
         
        del_list=[]#del_list=[]
        not_in_df_=[]
        
        for n,package in dic_columns.items():
            if n in a:
                for col in package:
                    if col in columns:
                        del_list.append(col)
                    else:
                        not_in_df_.append(col)
                        #pc=pc.drop([col],axis=1)
                        
                        #pc=pc.drop([col],axis=1)
        #else:
        #        input("Wrong input. Enter any key to try again..")
        
        
        return del_list,not_in_df_
#--------------------------------------------------------------------------------------------- 
    dic_columns={0:['treatment','index'],
                     1:['nb_i_0','nb_i_1','nb_i_2','nb_d_0','nb_d_1','nb_d_2'],
                     2:['nba_d_5','nba_d_10','nba_d_15'],
                     3:['cl_idx_5','cl_idx_7.5','cl_idx_10','cl_idx_15','cl_idx_20','cld'],
                     4:['cont_tot','displ_tot','dvz_tot','dvy_tot'],
                     5:['elong', 'flatness','treatment', 'cohort', 'eigval_0', 'eigval_1', 'eigval_2'],
                     6:['c2_mean', 'c2_max'],
                    7:['mov_class', 'movement'],
                    8:['position','inside_injury', 'height', 'z_pos', 'zz', 'zled'],
                    9:['dvz_s','cont_s']}
    #global exp_series,inh_order,inh_labels
    
    #print("These dataframes are available in your directory")
    
    df_=[]
    arr_txt = [x for x in os.listdir() if x.endswith("dataframe")]
    for c, value in enumerate(arr_txt,0):
        print(c, value) 
    col_vars=input("Choose the dataframes you want to use separeted by spaces from the list below:")
    #print("Enter your choice as integers separeted by spaces: ")
    flist = [int(x) for x in col_vars.split()]
    
    clear_output(wait=False)    
    print_menu(dic_columns)    # Displays menu
        #choice = input("Enter your choice as a list of integers: ")
    
    a = [int(x) for x in input("Enter your choice as integers separeted by spaces: ").split()]
    
    clear_output(wait=False)
    
    for n_df,fi in enumerate(flist):
        pc=pd.read_pickle(arr_txt[fi])
        if n_df==0:
            #print("These Columns are present in your dataframes")
            columns=pc.columns
            #print(columns)
        del_list,not_in_df_=choices(columns,n_df,a,dic_columns)
        pc=pc.drop(del_list,axis=1)
        print(arr_txt[fi])
        print(f'Columns deleted: {del_list}',
              f'\nColumns not in original dataframe: {not_in_df_}')
        clear_output(wait=True)
        #print(f'Columns not in original dataframe: {not_in_df_}')
        df_.append(pc)        
    pc=pd.concat(df_, ignore_index=True,names='pid').reset_index()
    pc.index.name = 'pid'
    
    #clear_output(wait=False)
    choice = input('Do you only want to analyze tracked plts? (y/n)')
    if choice =='y':
        print('Set tracking threshold as integer')
        thr_tr=int(input())
        pc=pc[pc.nrtracks>thr_tr]
    #clear_output(wait=False)
    print(f'RESULTING DATAFRAME\nNo of columns: {pc.shape[1]} \nNo of rows: {pc.shape[0]}',
          f'\nMemory use: {np.round(sys.getsizeof(pc)/1000000,1)} Mb')
    
    return pc#del_list

In [ ]:
def get_menu_choice():
    def print_menu():       # Your menu design here
        print(30 * "-", "Choose experimental cohort to run", 30 * "-")
        print("1. Inhibitor cohort ")
        print("2. PAR4 cohort ")
        print("3. Thrombin cohort ")
        print("4. Custom cohort")        
        print("5. Exit from the script ")
        print(73 * "-")
    
    global exp_series,inh_order,inh_labels
    loop = True
    int_choice = -1

    while loop:          # While loop which will keep going until loop = False
        print_menu()    # Displays menu
        choice = input("Enter your choice [1-4]: ")

        if choice == '1':
            exp_series = 'Inhibitor cohort'
            inh_order=['saline','dmso','biva','sq','cang']
            inh_labels=['Saline','DMSO','Bivalirudin','Sq','Cangrelor']

            loop = False
        elif choice == '2':
            exp_series = 'PAR4 cohort'
            inh_order=['ctrl','par4+-','par4-+','par4--']
            inh_labels=['Control','PAR4 +/-','PAR4 -/+','PAR4-/-']
            loop = False
        elif choice == '3':
            exp_series = 'Thrombin cohort'
            inh_order=['saline','biva','par4--','par4--biva']
            inh_labels=['Saline','Bivalirudin','PAR4 -/-','PAR4 -/- + Bivalirudin']
            loop = False
        elif choice == '4':
            choice = ''
            while len(choice) == 0:
                exp_series = input("Enter a single filename of a conf file: ")
            loop = False
        elif choice == '5':
            int_choice = -1
            print("Exiting..")
            loop = False  # This will make the while loop to end
        else:
            # Any inputs other than values 1-4 we print an error message
            input("Wrong menu selection. Enter any key to try again..")
    create_params()
   # return [int_choice, choice]

#Inhibitor orders for graphs 
#-------------------------------------------------------
#inh_order1=['ctrl','biva','par4--','cang','par4--biva']
#inh_order=['ctrl','saline','biva','par4--','cang','par4--biva']
#inh_order1=['saline','biva','par4--','cang','par4--biva']
#inh_order_thrombin=['saline','biva','par4--','par4--biva']

#inh_order2=['saline','dmso','biva','sq','cang']
#Inhibitor labels for graphs 
#-------------------------------------------------------
#inh_labels=['Control','Saline','Bivalirudin','PAR4-/-','Cangrelor','PAR4-/- + Bivalirudin']
#
#


# Create new dataframes

In [1]:
def create_col_dics():
    print(f'Categories of columns in dataframe for selection:\n{73 * "-"}')
#col_index=[0]
    dic_list=[]
    dic_names=[]
    for nr,(key,c_list) in enumerate(dic_df_cols.items()):
        print(f'{key}\n{73 * "-"}')
    #i_col,col_list = [ic for i,c in enumerate(c_list)]
    #print(i_col,col_list)
    #i_list,col_list=[],[]
        #dic_name='dic_'+key
        #print(dic_name)
        key=dict()
        for i_col,col in enumerate(c_list):
            key[i_col]=col
        print(key)
        dic_list.append(key)
        dic_names.append(dic_name)
        #dic_name=key
        #i_list.append(i_col)
        #col_list.append(col)
    #print(i_list,col_list)
        print(f'{73 * "-"}')
    #return(dic_list,dic_names)

In [ ]:
def view_col_dics():
    print(f'Categories of columns in dataframe for selection:\n{73 * "-"}\t')
#col_index=[0]
    dic_list=[]
    dic_names=[]
    for nr,(key,c_list) in enumerate(dic_df_cols.items()):
        print(key)
        print(f'{73 * "-"}')
        print(f'{c_list}\n{73 * "-"}\n')

In [ ]:
def build_df_lists(df_var_list):
#for cat_names in dic_names:
#for name,dic in zip(dic_names,dic_list):
#    dic_names
    from IPython.display import clear_output    
    clear_output(wait=False)

    df_=[]
    #df_cols=['pid', 'path', 'inh',]
    df_cols=[]
    col_list = [item for sublist in df_var_list for item in sublist]
    #print(col_list)
    #print(col_list,col_dict,'******')
    #print(col_dict[col_list[0]])
 
    p = Path('./Dataframes').glob('**/*')
    paths = [x for x in p if x.is_file()]
    list1=[path for inh in inh_order for path in paths if inh in path.name]
    path_list=list(set(list1))
    
    for n_df,fi in enumerate(path_list):
        dfi=pd.read_pickle(fi)
        dfi_col=['path', 'inh','particle']
        for col in col_list:
            if col in dfi.columns:
                dfi_col.append(col)      
        df_.append(dfi.loc[:,dfi_col])
    pc=pd.concat(df_, ignore_index=True).reset_index()#,names='pid'     
    
    
    
    
    if 'minute' in pc.columns:
        pc.loc[:,'minute'] = pd.cut(pc['time'], 10, labels=np.arange(1,11,1))
    if 'nrtracks' in pc:    
        pc=nrtracks_menu(pc)
    
    
    pc=new_exp_ids(pc)
    pc=outliers_menu(pc)
    plot_menu()
    
    pc=pc.drop(['level_0','index'],axis=1).reset_index()
    pc.index.name = 'pid'
    #pc.inh_exp_id.unique()
    
    #clear_output(wait=False)
    print(f'Treatments = {pc.inh.unique()}') #Paths included = {pc.path.unique()}\n
    print(f'RESULTING DATAFRAME\nNo of columns: {pc.shape[1]} \nNo of rows: {pc.shape[0]}',
          f'\nMemory use: {np.round(sys.getsizeof(pc)/1000000,1)} Mb')
    
    return pc

In [ ]:
def new_exp_ids(pc):
    pc["exp_id"] = pc.groupby(["path"]).ngroup()
    for inh in pc.inh.unique():
        pc.loc[pc.inh==inh,'inh_exp_id']=pc[pc.inh==inh].groupby('path').ngroup()#grouper.group_info[0]
    return pc

In [ ]:
def nrtracks_menu(pc):
    choice = input('Do you only want to analyze tracked plts? (y/n)')
    if choice =='y':
        print('Set tracking threshold as integer')
        thr_tr=int(input())
        pc=pc[pc.nrtracks>thr_tr].reset_index()
    return pc

In [ ]:
def outliers_menu(pc):
    choice = input('Remove Outliers? (y/n)')
    if choice =='y':
        outliers=pd.read_csv('df_outliers')
        pc=pc[~pc.path.isin(outliers.path)]
    return pc

In [ ]:
 def plot_menu():
    global save_figs,results_folder,plot_formats,save_inh_names
    choice = input('Automatically save plots? (y/n)')
    if choice =='y':
        save_figs=True
        results_folder = input('Write name of folder for plots')
        makedir(results_folder)
        
        plot_formats = input('Choose file format for plots:\nWrite "p" for .png, "s" for .svg or "b" for both')
        
        save_names = input('Use treatment names in filenames? (y/n)')
        if choice =='y':
            save_inh_names=True
        else:
            save_inh_names=False
    
    if choice =='n':
        save_figs=False
        save_inh_names=False
    

In [ ]:
def build_df_lists1(df_var_list):
#for cat_names in dic_names:
#for name,dic in zip(dic_names,dic_list):
#    dic_names
    from IPython.display import clear_output    
    clear_output(wait=False)

    df_=[]
    #df_cols=['pid', 'path', 'inh',]
    df_cols=[]
    col_list = [item for sublist in df_var_list for item in sublist]
    #print(col_list)
    #print(col_list,col_dict,'******')
    #print(col_dict[col_list[0]])
 
    arr_txt = [x for x in os.listdir() if x.endswith("dataframe")]
    for c, value in enumerate(arr_txt,0):
        print(c, value) 
    col_vars=input("Choose the dataframes you want to use separated by spaces from the list below:")
    #print("Enter your choice as integers separeted by spaces: ")
    flist = [int(x) for x in col_vars.split()]
    
    
    #print(f'Including columns:{df_cols}')
    #clear_output(wait=False)
    for n_df,fi in enumerate(flist):
        dfi=pd.read_pickle(arr_txt[fi])
        dfi_col=['pid', 'inh','particle']
        for col in col_list:
            if col in dfi.columns:
                dfi_col.append(col)      
        df_.append(dfi.loc[:,dfi_col])
    pc=pd.concat(df_, ignore_index=True,names='pid').reset_index()
    pc.index.name = 'pid'
    
    if 'nrtracks' in pc:
        choice = input('Do you only want to analyze tracked plts? (y/n)')
        if choice =='y':
            
            
            print('Set tracking threshold as integer')
            thr_tr=int(input())
            pc=pc[pc.nrtracks>thr_tr]
    #clear_output(wait=False)
    print(f'RESULTING DATAFRAME\nNo of columns: {pc.shape[1]} \nNo of rows: {pc.shape[0]}',
          f'\nMemory use: {np.round(sys.getsizeof(pc)/1000000,1)} Mb')
    return pc

In [ ]:
def build_df():
    from IPython.display import clear_output    

#--------------------------------------------------------------------------------------------- 
    dic_columns={'positions1':['x_s', 'ys', 'zs'],
                 'positions2':['position','inside_injury', 'height'],
                 'positions3':['depth', 'z_pos', 'zz', 'zled','zf','minute'],
                 'dist_c':['dist_c', 'dist_cz'],
                 'int_c0':['c0_mean', 'c0_max'],
                 'int_c1':['c1_mean', 'c1_max'],
                 "int_c2":['c2_mean', 'c2_max'],
                 "plt_shape":['elong', 'flatness','treatment', 'cohort', 'eigval_0', 'eigval_1', 'eigval_2'],
                 "nearest neighbours":['nb_i_0','nb_i_1','nb_i_2','nb_d_0','nb_d_1','nb_d_2'],
                 'average dist nearest neighbours':['nba_d_5','nba_d_10','nba_d_15'],
                 "DBSCAN":['cl_idx_5','cl_idx_7.5','cl_idx_10','cl_idx_15','cl_idx_20','cld'],
                 "xtra_tracking":['cont_tot','displ_tot','dvz_tot',],#'dvy_tot'
                 'mov_1':['dvx', 'dvy', 'dvz', 'dv','stab'],
                 'mov_2':['mov_class', 'movement'],
                 'mov_3':['cont', 'cont_p'],
                 'mov_s':['dvz_s','cont_s'],
                 'info':['inj', 'exp', 'date','mouse'] 
                         #'treatment', 'cohort', ]
        
                 
    }
    #global exp_series,inh_order,inh_labels
    
    #print("These dataframes are available in your directory")
    df_=[]
    df_cols=[['pid', 'path', 'frame','time','inh','minute','nrtracks','tracknr']]
    arr_txt = [x for x in os.listdir() if x.endswith("dataframe")]
    for c, value in enumerate(arr_txt,0):
        print(c, value) 
    col_vars=input("Choose the dataframes you want to use separeted by spaces from the list below:")
    #print("Enter your choice as integers separeted by spaces: ")
    flist = [int(x) for x in col_vars.split()]
    
    for n,(i,c_list) in enumerate(dic_columns.items()):
        c=input(f'Include :{c_list}? (y/n)')
        if c=='y':
            df_cols.append(c_list)
    #print(df_cols)
    df_cols = [item for sublist in df_cols for item in sublist]
    print(f'Including columns:{df_cols}')
    #clear_output(wait=False)
    for n_df,fi in enumerate(flist):
        dfi=pd.read_pickle(arr_txt[fi])
        df_.append(dfi.loc[:,df_cols])
    pc=pd.concat(df_, ignore_index=True,names='pid').reset_index()
    pc.index.name = 'pid'
    
    choice = input('Do you only want to analyze tracked plts? (y/n)')
    if choice =='y':
        print('Set tracking threshold as integer')
        thr_tr=int(input())
        pc=pc[pc.nrtracks>thr_tr]
    #clear_output(wait=False)
    print(f'RESULTING DATAFRAME\nNo of columns: {pc.shape[1]} \nNo of rows: {pc.shape[0]}',
          f'\nMemory use: {np.round(sys.getsizeof(pc)/1000000,1)} Mb')
    
    return pc#del_list
    
    